# Customer churn prediction using Fuzzy Inference Systems

*Skeleton code for using FIS in customer churn. Example for car sharing fleet management and maintenance tickets*

In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Read csv
def read_sensor(path):
    return np.genfromtxt(path, delimiter = ',')

In [ ]:
# data location set 
pathActiveCarsInFleet = 'customer_nrActiveCarsInFleet_raw.csv'
pathMaintenanceTickets = 'customer_nrMaintenanceTickets_raw.csv'

# load data
dataActiveCarsInFleet = read_sensor(pathActiveCarsInFleet)
dataMaintenanceTickets = read_sensor(pathMaintenanceTickets)

In [ ]:
# random seed init
np.random.seed(0)
# build the fuzzy inference variables
# inputs
nrActiveCarsInFleet = ctrl.Antecedent(universe=np.arange(min(dataActiveCarsInFleet), max(dataActiveCarsInFleet), 1), label='nrActiveCarsInFleet')
nrMaintenanceTickets = ctrl.Antecedent(universe=np.arange(min(dataMaintenanceTickets), max(dataMaintenanceTickets), 1), label='nrMaintenanceTickets')
# output
churnProbability = ctrl.Consequent(universe=np.arange(0, 101, 1), label='churnProbability')

In [ ]:
# parametrize the variables membership functions
# inputs
nrActiveCarsInFleet.automf(number=3, variable_type='number', names=['low', 'medium', 'high'])
nrMaintenanceTickets.automf(number=3, variable_type='number', names=['low', 'medium', 'high'])
# output
churnProbability['low'] = fuzz.trimf(x=churnProbability.universe, abc=[0, 0, 50])
churnProbability['medium'] = fuzz.trimf(x=churnProbability.universe, abc=[0, 50, 100])
churnProbability['high'] = fuzz.trimf(x=churnProbability.universe, abc=[50, 100, 100])

In [ ]:
# add the Business Intelligence rules for churning
rule1 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['low'] & nrMaintenanceTickets['low'],
    consequent=churnProbability['low']
)

rule2 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['low'] & nrMaintenanceTickets['medium'],
    consequent=churnProbability['medium']
)

rule3 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['medium'] & nrMaintenanceTickets['low'],
    consequent=churnProbability['low']
)

rule4 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['medium'] & nrMaintenanceTickets['medium'],
    consequent=churnProbability['medium']
)

rule5 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['high'] & nrMaintenanceTickets['low'],
    consequent=churnProbability['low']
)

rule6 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['low'] & nrMaintenanceTickets['high'],
    consequent=churnProbability['high']
)


rule7 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['medium'] & nrMaintenanceTickets['high'],
    consequent=churnProbability['high']
)

In [ ]:
# build the inference engine
churnProbability_ctrl = ctrl.ControlSystem(rules=[rule1, rule2, rule3, rule4, rule5, rule6, rule7])

# build 
churnProbability = ctrl.ControlSystemSimulation(control_system=churnProbability_ctrl)
churnProbability.reset()

In [ ]:
# predict
for i in range(len(dataActiveCarsInFleet)):
    churnProbability.inputs({
        'nrActiveCarsInFleet': dataActiveCarsInFleet[i],
        'nrMaintenanceTickets': dataMaintenanceTickets[i]
    })
    churnProbability.compute()
    churnProbability.output['churnProbability']